# Logistic Regression Baseline Model

In [2]:
import csv
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression as LogisticRegression

### Import data and set up model

In [3]:
x_test = pd.read_csv('ImputedX_test.csv')
x_train = pd.read_csv('ImputedX_train.csv')
y_test = pd.read_csv('Y_test.csv')
y_train = pd.read_csv('Y_train.csv')
columns = y_train.columns

In [4]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
y_train.head()

(11225, 813) (2807, 813) (11225, 17) (2807, 17)


,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Mystery,Romance,Science Fiction,Thriller,War,Western
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
x_train = x_train.values
x_test = x_test.values
y_train = y_train.values
y_test = y_test.values

In [6]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [7]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score as f1_score
from sklearn.metrics import confusion_matrix
from sklearn.multiclass import OneVsRestClassifier

In [8]:
log_reg = LogisticRegression(penalty='l2', 
                           dual=False, 
                           tol=0.0001, 
                           C=1.0, 
                           fit_intercept=True, 
                           intercept_scaling=1, 
                           class_weight="balanced", 
                           random_state=None, 
                           solver='liblinear', 
                           max_iter=100, 
                           multi_class='ovr', 
                           n_jobs=1)

## Tune all genres simultaneously using sklearn's OneVsRest multi-label classification

In [9]:
ovr = OneVsRestClassifier(log_reg,n_jobs=3)

In [10]:
#parameters = {'C':[1e-15, 1e-10, 1e-5, 0.1, 1]}
parameters = {'estimator__penalty':['l2'], 'estimator__C': [1e-15, 1e-10, 0.1, 1]}

ovr_model = GridSearchCV(ovr, 
              parameters, 
              scoring="f1", 
              fit_params=None, 
              n_jobs=3, 
              iid=True, 
              refit=True, 
              cv=3, 
              verbose=0, 
              pre_dispatch='2*n_jobs', 
              error_score='raise')
ovr_model.fit(x_train, y_train)

//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda

GridSearchCV(cv=3, error_score='raise',
       estimator=OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
          n_jobs=3),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'estimator__penalty': ['l2'], 'estimator__C': [1e-15, 1e-10, 0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='f1', verbose=0)

## Perform tuning separately for each genre

In [53]:
#parameters = {'C':[1e-15, 1e-10, 1e-5, 0.1, 1]}
parameters = {'estimator__penalty':['l2'], 'estimator__C': [1e-15, 1e-10, 0.1, 1]}
best_models = []
for i in range(y_train.shape[1]):
    grid_model = GridSearchCV(ovr, 
              parameters, 
              scoring="f1", 
              fit_params=None, 
              n_jobs=4, 
              iid=True, 
              refit=True, 
              cv=3, 
              verbose=0, 
              pre_dispatch='2*n_jobs', 
              error_score='raise')
    grid_model.fit(x_train, y_train[:,i])
    grid_model.best_estimator_.fit(x_train, y_train[:,i])
    best_models.append(grid_model.best_estimator_)
    print f1_score(y_train[:,i], grid_model.best_estimator_.predict(x_train),average='binary')
    f1score = f1_score(y_test[:,i], grid_model.best_estimator_.predict(x_test), average='binary')
    print "Final test F1 score: ", f1score
    test_scores.append(f1score)

//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.

0.397630697914
Final test F1 score:  0.269396551724


//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.

0.282769726248
Final test F1 score:  0.141496598639


//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.

...

0.375955170657
Final test F1 score:  0.265159301131


//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.

0.33650329188
Final test F1 score:  0.152941176471


//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
//anaconda/envs/py27/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.

0.484947111473
Final test F1 score:  0.396449704142


In [59]:
best_models

[OneVsRestClassifier(estimator=LogisticRegression(C=0.1, class_weight='balanced', dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
           n_jobs=3),
 OneVsRestClassifier(estimator=LogisticRegression(C=0.1, class_weight='balanced', dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
           n_jobs=3),
 OneVsRestClassifier(estimator=LogisticRegression(C=1, class_weight='balanced', dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
           n_jobs=3),
 OneVsRestClassifi

### Look at top 5 most important (positively correlated) predictors for each genre!

In [30]:
x_train_raw = pd.read_csv('ImputedX_train.csv')
features = x_train_raw.columns
best_estimator = ovr_model.best_estimator_
for index, row in enumerate(best_estimator.coef_):
    print("\n", columns[index])
    most_important_predictors = row.argsort()[-5:][::-1]
    print(features[most_important_predictors].values)


 Action
['popularity' 'languages_serbo' 'languages_japanese' 'languages_russian'
 'languages_greek']

 Adventure
['green_pixel' 'countries_japan' 'countries_brazil' 'languages_tagalog'
 'languages_english']

 Animation
['countries_japan' 'cast_melblanc' 'green_pixel' 'countries_southkorea'
 'languages_thai']

 Comedy
['overview_comedy' 'red_pixel' 'director_davefleischer' 'color info_path'
 'cast_hankbell']

 Crime
['languages_japanese' 'overview_murder' 'languages_serbo' 'languages_hindi'
 'budget']

 Documentary
['rating' 'overview_documentary' 'color info_color' 'writer_nan'
 'languages_english']

 Drama
['overview_drama' 'runtime' 'votes' 'rating' 'writer_howardj']

 Family
['green_pixel' 'overview_children' 'director_chuckjones'
 'cast_jeffreysayre' 'vote_count']

 Fantasy
['languages_tagalog' 'countries_japan' 'cast_shea' 'countries_southkorea'
 'vote_average']

 History
['rating' 'red_pixel' 'cast_nan' 'overview_high' 'countries_czechoslovakia']

 Horror
['color info_color' 'la

## Perform final evaluation

### F1 scores for simulataneous tuning

In [17]:
f1_score(y_test, ovr_model.best_estimator_.predict(x_test), pos_label=1, average=None)

array([ 0.26939655,  0.1414966 ,  0.42285714,  0.48171075,  0.17878427,
        0.57228315,  0.58528139,  0.20779221,  0.08831909,  0.03317536,
        0.35322196,  0.10818713,  0.26804124,  0.20221607,  0.2651593 ,
        0.15340909,  0.3964497 ])

In [18]:
ovr_f1_scores = f1_score(y_test, ovr_model.predict(x_test), pos_label=1, average=None)
print("F1 scores: ", ovr_f1_scores, "\n")
print("Average F1 score: ", ovr_f1_scores.mean())

F1 scores:  [ 0.26939655  0.1414966   0.42285714  0.48171075  0.17878427  0.57228315
  0.58528139  0.20779221  0.08831909  0.03317536  0.35322196  0.10818713
  0.26804124  0.20221607  0.2651593   0.15340909  0.3964497 ] 

Average F1 score:  0.278104763963


### F1 scores for separate tuning

In [55]:
print "All F1 scores: ", test_scores, "\n"
print "Average F1 score: ", np.array(test_scores).mean()

All F1 scores:  [0.2693965517241379, 0.1414965986394558, 0.42285714285714282, 0.48171074845244793, 0.17878426698450534, 0.57228315054835488, 0.2693965517241379, 0.1414965986394558, 0.42485549132947975, 0.48171074845244793, 0.17878426698450534, 0.57228315054835488, 0.5852813852813854, 0.20648967551622419, 0.083694083694083696, 0.033175355450236962, 0.3532219570405728, 0.10818713450292397, 0.26804123711340205, 0.20221606648199447, 0.26515930113052416, 0.15294117647058825, 0.39644970414201186] 

Average F1 score:  0.295213580161


### Hamming loss for simulaneous tuning

In [56]:
from sklearn.metrics import hamming_loss

In [57]:
hamming_loss(y_test, model.predict(x_test))

0.20876380477377984

### Hamming loss for separate tuning

In [58]:
y_test_multilabel = np.empty((y_test.shape[0], len(best_models)))
for index, m in enumerate(best_models):
    y_test_multilabel[:,index] = m.predict(x_test)

print(hamming_loss(y_test[:,0:len(best_models)], y_test_multilabel))

0.208093212347


### Results

- Simultaneous tuning and separate tuning are comparable for F1 scores and hamming loss. 
- Performance seems relatively poor, with F1 scores less than 0.3